# Using Tools
This notebook demonstrates how we can use tools to gather information and provide more accurate responses. We'll build a weather-checking mock as an example.

## What we'll learn:
- Basic interaction with Language Models (LLM)
- How to create and use tools with AI
- The complete flow of an AI agent using tools
- Understanding the message flow in a tool-enabled conversation

### Setup

In [1]:
import json
from dotenv import load_dotenv
from lib.messages import UserMessage, SystemMessage, ToolMessage  # Different message types
from lib.tooling import tool  # Tool decorator for creating AI tools
from lib.llm import LLM  # Our Language Model wrapper



In [2]:
load_dotenv(override=True)

True

In [3]:
chat_model = LLM()

## Basic LLM Interaction
Before we dive into tools, let's understand how to interact with our Language Model in its simplest form. 
There are two main ways to communicate with the model:

In [4]:
# Method 1: Simple single-turn query
response = chat_model.invoke("What is an AI Agent?")
print("Single Query Response:\n", response.content)

Single Query Response:
 An AI agent is a system or entity that uses artificial intelligence techniques to perceive its environment, make decisions, and take actions to achieve specific goals. AI agents can operate autonomously or semi-autonomously and can be found in various applications, ranging from simple rule-based systems to complex machine learning models.

Key characteristics of AI agents include:

1. **Perception**: AI agents can gather information from their environment through sensors or data inputs. This could involve processing visual data, audio signals, or other forms of input.

2. **Decision-Making**: Based on the information they perceive, AI agents can analyze the data and make decisions. This can involve reasoning, planning, and learning from past experiences.

3. **Action**: After making decisions, AI agents can take actions to interact with their environment. This could involve physical actions (like a robot moving) or digital actions (like a software agent executin

In [5]:
# Method 2: Multi-turn conversation with specific roles
messages = [
    SystemMessage(content="You're an OpenAI API specialist"),
    UserMessage(content="What is Function Calling?")
]
response = chat_model.invoke(messages)
print("\nStructured Conversation Response:\n", response.content)


Structured Conversation Response:
 Function calling is a programming concept where a function is executed or invoked in order to perform a specific task or computation. In programming, a function is a reusable block of code that takes inputs (known as parameters or arguments), processes them, and often returns an output.

### Key Aspects of Function Calling:

1. **Definition**: A function must be defined before it can be called. This definition includes the function's name, parameters, and the code that will be executed when the function is called.

2. **Parameters**: Functions can take parameters, which are values passed to the function when it is called. These parameters allow the function to operate on different data.

3. **Return Value**: Functions can return a value after execution. This return value can be used in further computations or operations.

4. **Syntax**: The syntax for calling a function varies by programming language but generally involves the function's name followe

## Building an AI Tool
Now let's make our AI more capable by giving it a tool to check the weather. 
This demonstrates how we can extend AI capabilities beyond just conversation.

### Understanding the Tool Structure:
1. We use the `@tool` decorator to mark a function as an AI tool
2. The tool needs clear documentation and typed parameters
3. The tool should return structured data

In [6]:
@tool
def get_weather(city: str):
    """Get the current temperature for a city.
    
    Args:
        city (str): Name of the city to check weather for
        
    Returns:
        dict: Contains temperature information for the requested city
    """
    # In a real application, this would call a weather API
    mock_weather = {
        "São Paulo": "28°C",
        "Oslo": "-3°C",
        "New York": "15°C",
        "Tokyo": "22°C"
    }
    return {"temperature": mock_weather.get(city, "Unknown")}

In [7]:
# Bind the tool to an LLM
chat_model_with_tools = LLM(tools=[get_weather])

## Understanding the Tool Usage Flow
Let's break down how the AI uses tools step by step:

1. User asks a question about weather
2. AI recognizes the need to use the weather tool
3. AI makes a tool call
4. Tool executes and returns results
5. AI processes the tool's response
6. AI formulates a natural language response

Let's see this in action:

In [8]:
# Set up our system with clear instructions
messages = [
    SystemMessage(
        content="You are a helpful assistant that can access a tool to get current temperature " 
                "for cities. Use the tool whenever someone asks about the weather or temperature " 
                "in a specific location. Infor the user if you don't know the answer."
    ),
    UserMessage(content="How cold is it in Oslo?")
]

In [10]:
# AI decides to use the weather tool
ai_message = chat_model_with_tools.invoke(messages)
ai_message

AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')])

In [11]:
# Check messages structure
messages.append(ai_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')])]

In [12]:
# Tool call id will be required later when creating the ToolMessage
tool_call_id = messages[-1].tool_calls[0].id
tool_call_id

'call_yGVMq3Mdk4jofN1KlYEcbcLQ'

In [13]:
# Extract the arguments
args = json.loads(messages[-1].tool_calls[0].function.arguments)
args

{'city': 'Oslo'}

In [14]:
# Execute the tool with the AI's requested parameters
tool_result = get_weather(**args)
tool_result

{'temperature': '-3°C'}

In [15]:
# Create a tool response message
tool_message = ToolMessage(
    content=tool_result["temperature"], 
    tool_call_id=tool_call_id, 
    name="get_weather"
)
tool_message

ToolMessage(content='-3°C', role='tool', tool_call_id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', name='get_weather')

In [16]:
# Check messages structure
messages.append(tool_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')]),
 ToolMessage(content='-3°C', role='tool', tool_call_id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', name='get_weather')]

In [17]:
# Let AI formulate final response
ai_message = chat_model_with_tools.invoke(messages)
ai_message

AIMessage(content='The current temperature in Oslo is -3°C.', role='assistant', tool_calls=None)

In [18]:
# Check messages structure
messages.append(ai_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')]),
 ToolMessage(content='-3°C', role='tool', tool_call_id='call_yGVMq3Mdk4jofN1KlYEcbcLQ', name='get_weather'),
 AIMessage(content='The current temperature in Oslo is -3°C.', role='assistant', tool_calls=None)]